In [1]:
from json import  dumps
from pathlib import Path
from yaml import dump as y_dump, FullLoader
from datetime import date

today = date.today().isoformat()
my_id = 'my_id'
my_profile = 'my_profile'
my_resource = 'my_resource'

import pandas
# import scopes table
sub_table_path = Path(r'/Users/ehaas/Documents/FHIR/USCDI5-Sandbox/input/data/scopes.csv')
out_path = Path(r'/Users/ehaas/Documents/FHIR/USCDI5-Sandbox/input/resources-yaml')
# convert to df
df = pandas.read_csv(sub_table_path, index_col="topic")
df.head()

,row,topic_path,topic_url,topic_conformance,profile_title,page_path,data_element,resource_type,category_1,category_1_conformance,category_2,category_2_conformance,category_3,category_3_conformance,category_4,category_4_conformance,category_5,category_5_conformance,category_6,category_6_conformance
topic,,,,,,,,,,,,,,,,,,,,
NaN,1,NaN,NaN,MAY,US Core AllergyIntolerance Profile,StructureDefinition-us-core-allergyintolerance...,allergy and intolerance,AllergyIntolerance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,2,NaN,NaN,MAY,US Core CarePlan Profile,StructureDefinition-us-core-careplan.html,care plan,CarePlan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,3,NaN,NaN,MAY,US Core CareTeam Profile,StructureDefinition-us-core-careteam.html,careteam,CareTeam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,4,NaN,NaN,MAY,US Core Condition Encounter Diagnosis Profile,StructureDefinition-us-core-condition-encounte...,encounter diagnosis,Condition,http://terminology.hl7.org/CodeSystem/conditio...,SHALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
US Core Problems and Health Concerns,5,SubscriptionTopic-us-core-problems-and-health-...,http://hl7.org/fhir/us/core/SubscriptionTopic/...,SHALL,US Core Condition Problems and Health Concerns...,StructureDefinition-us-core-condition-problems...,problems and health concern,Condition,http://terminology.hl7.org/CodeSystem/conditio...,SHALL,http://hl7.org/fhir/us/core/CodeSystem/conditi...,SHALL,foo,NaN,bar,NaN,NaN,NaN,NaN,NaN


In [2]:
if __name__ == "__main__":
    for i in df.index:
        if i==i: # not NaN
            i_strip = i.strip() 
            print(f"i = {i_strip}, df.loc[i]['profile_title'] = {df.loc[i]['profile_title']},\
                \n df.loc[i]['resource_type'] = {df.loc[i]['resource_type']}")
            my_id = i_strip.lower().replace(' ','-')  # get topic column and make it id
            my_profile = df.loc[i]['profile_title']  # get profile_title column
            my_resource = df.loc[i]['resource_type']  # get resource_type column
            
            st_template = { # f string do runtime replacement so easiest if inline or a function call
            "resourceType": "SubscriptionTopic",
            "id": f"{my_id}",
            "url": f"http://hl7.org/fhir/us/core/SubscriptionTopic/{my_id}",
            "version": "0.0.0",
            "name": f"{i.title().replace('Us Core','USCore').replace(' ','').replace('-','')}SubscriptionTopic",
            "title": f"{i_strip} Subscription Topic",
            "status": "active",
            "date": today,
            "description": f"This topic allows subscribers to receive notifications for the {i_strip} events represented by US Core {my_profile}, including create, update, and delete interactions.",
            "resourceTrigger": [
                {
                    "description": f"Trigger on {i_strip} events",
                    "resource": my_resource,  # profile or resource?
                    "supportedInteraction": [
                        "create",
                        "update",
                        "delete"
                    ],
                    "fhirPathCriteria": "%previous.empty() or %current.empty() or %previous != %current",
                }
            ],
            "eventTrigger": [
                {
                    "description": f"Trigger on patient-related clinical {i_strip.replace('US Core','').lower()} events",
                }
            ],
            "canFilterBy": [
                        {
                            "description": "Filter by patient",
                            "filterParameter": "patient"
                        }
                    ]
            }

            # print(dumps(st_template,indent=2))

            #update the data frame with topic_path and topic_url
            # print(f"df.loc[i]['topic_path'] = {f'SubscriptionTopic-{my_id}.html'}")
            # print(f"df.loc[i]['url'] = {st_template['url']}")
            df.at[i,'topic_path'] = f'SubscriptionTopic-{my_id}.html'
            df.at[i,'topic_url'] = st_template['url']

            #convert to Yaml and write to out
            file_path = out_path /f'SubscriptionTopic-{my_id}.yml'
            print(f'\nwriting to {file_path}\n')
            print(y_dump(st_template, sort_keys=False))
            # print(f'resource_type = {st_template["trigger"][0]["resource"]}')
            file_path.write_text(y_dump(st_template, sort_keys=False), encoding="utf-8")

i = US Core Problems and Health Concerns, df.loc[i]['profile_title'] = US Core Condition Problems and Health Concerns Profile,                
 df.loc[i]['resource_type'] = Condition

writing to /Users/ehaas/Documents/FHIR/USCDI5-Sandbox/input/resources-yaml/SubscriptionTopic-us-core-problems-and-health-concerns.yml

resourceType: SubscriptionTopic
id: us-core-problems-and-health-concerns
url: http://hl7.org/fhir/us/core/SubscriptionTopic/us-core-problems-and-health-concerns
version: 0.0.0
name: USCoreProblemsAndHealthConcernsSubscriptionTopic
title: US Core Problems and Health Concerns Subscription Topic
status: active
date: '2024-08-16'
description: This topic allows subscribers to receive notifications for the US Core
  Problems and Health Concerns events represented by US Core US Core Condition Problems
  and Health Concerns Profile, including create, update, and delete interactions.
resourceTrigger:
- description: Trigger on US Core Problems and Health Concerns events
  resource: 

In [3]:
#save updated df as CS
df.to_csv(sub_table_path)